In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [2]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ',
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = str.strip('\n')
    str = str.strip(' ')

    return str

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./STTRResult/04_STTR_phase2.csv', index=False, encoding="utf-8")
    
def ReadFiles(Directory_path):  # Read all the html files
    path = Directory_path  # The directory
    file_names = os.listdir(path)  # 得到文件夹下的所有文件名称
    files_position = []
    for file_name in file_names:
        position = path + "/" + file_name
        # print(position)
        files_position.append(position)

    print("Total number of HTML files: ", len(files_position))
    return files_position


def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

In [32]:
def SingleHTMLProcess(path):
    dic = {"Proposal Number": "", "Phase 1 Contract #": "", "Subtopic Title": "", "Proposal Title": "",
           "Small Business Concern_Firm": "", "Small Business Concern_Address": "", "Small Business Concern_Phone": "",
           "Principal Investigator_Name": "", "Principal Investigator_E-mail": "", "Principal Investigator_Address": "", "Is Principal Investigator US Citizen or Permanent Resident":"",
           "Principal Investigator_Phone": "", "Principal Investigator_Company": "",
           "Business Official_Name": "", "Business Official_E-mail": "", "Business Official_Address": "",
           "Business Official_Phone": "",
           "TRL_Begin": "", "TRL_End": "", "Technical Abstract": "", "Potential NASA applications": "",
           "Potential non-NASA applications": "","Technology Taxonomy Mapping":"", 'Research Topic':"", 'Research Institution Name':"", 'Research Institution Address':"",
          "Research Institution Phone":""}
    htmlfile = open(path, 'r', encoding='utf-8')
    html=htmlfile  
    bs = BeautifulSoup(html, "lxml")  # 缩进格式
    print(path)
    dic['Technical Abstract'] = filterHTMLstr(getTechAbstract(bs))
    dic['Potential NASA applications'] = filterHTMLstr(getNASAApplication(bs))
    dic['Potential non-NASA applications'] = filterHTMLstr(getNonNASAApplication(bs))
    dic['Proposal Number'] = filterHTMLstr(getProposalNumber(bs))
    dic['Subtopic Title'] = filterHTMLstr(getResearchSubTopicTitle(bs))
    dic['Proposal Title'] = filterHTMLstr(getProjectTitle(bs))
    dic['Phase 1 Contract #'] = filterHTMLstr(getPhase1Number(bs))
    smbAndRIInfo = getSMBAndRIInfo(bs)
    dic['Small Business Concern_Firm'] = filterHTMLstr(smbAndRIInfo[0])
    dic['Small Business Concern_Address'] = filterHTMLstr(smbAndRIInfo[1])
    dic['Small Business Concern_Phone'] = filterHTMLstr(smbAndRIInfo[2])
    dic['Research Institution Name'] = filterHTMLstr(smbAndRIInfo[3])
    dic['Research Institution Address'] = filterHTMLstr(smbAndRIInfo[4])
    dic['Research Institution Phone'] = filterHTMLstr(smbAndRIInfo[5])
    dic['Principal Investigator_Name'] = filterHTMLstr(getInvestigatorInfo(bs)[0])
    dic['Principal Investigator_E-mail'] = filterHTMLstr(getInvestigatorInfo(bs)[1])
    return dic

In [119]:
Directory_path = "../Datasets/04/sttr/phase2"
files_position = ReadFiles(Directory_path)
totaldata = MultipleFileProcess(files_position)
to_CSV(totaldata)

Total number of HTML files:  17
../Datasets/04/sttr/phase2/STTR-04-2-T5.01-9922.html
../Datasets/04/sttr/phase2/STTR-04-2-T8.02-9927.html
../Datasets/04/sttr/phase2/STTR-04-2-T6.01-9891.html
../Datasets/04/sttr/phase2/STTR-04-2-T8.01-9857.html
../Datasets/04/sttr/phase2/STTR-04-2-T9.01-9933.html
../Datasets/04/sttr/phase2/STTR-04-2-T2.02-9951.html
../Datasets/04/sttr/phase2/STTR-04-2-T1.01-9930.html
../Datasets/04/sttr/phase2/STTR-04-2-T4.02-9897.html
../Datasets/04/sttr/phase2/STTR-04-2-T2.02-9970.html
../Datasets/04/sttr/phase2/STTR-04-2-T7.01-9838.html
../Datasets/04/sttr/phase2/STTR-04-2-T1.02-9905.html
../Datasets/04/sttr/phase2/STTR-04-2-T8.01-9867.html


IndexError: list index out of range

# Test

In [120]:
path = "../Datasets/04/sttr/phase2/STTR-04-2-T8.01-9867.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式

In [61]:
Ps = bs.find_all("p")
NASAApplication = ""
for p in Ps:
    if "POTENTIAL NASA COMMERCIAL APPLICATIONS" in p.get_text():
        NASAApplication = p
NASAApplication

<p>
<b>POTENTIAL NASA COMMERCIAL APPLICATIONS </b>( Limit 1500 characters, approximately 150 words) <br/>
The proposed high performance Stokes polarization imaging system would find tremendous applications in NASA missions.  The development effort of this program will result in a light weight space applicable instrument which may well suited for NASAs molecular biological and medical studies under the microgravity environment. Potential application of this technology will allow microscopic imaging and biophysical measurements of cell functions, effects of electric or magnetic fields, photoactivation, and testing of drugs or biocompatible polymers on live tissues. The proposed portable device can also be a powerful and convenient tool for non-biological studies during the NASA mission. </p>

In [62]:
### Technical Abstract and Potential Commercial Application and NASA Application
def getTechAbstract(bs):
    Ps = bs.find_all("p")
    technicalAbstract = ""
    for p in Ps:
        if "TECHNICAL ABSTRACT" in p.get_text():
            technicalAbstract = p.get_text()
    return technicalAbstract

def getNonNASAApplication(bs):
    Ps = bs.find_all("p")
    nonNASAApplication = ""
    for p in Ps:
        if "POTENTIAL NON-NASA COMMERCIAL APPLICATIONS" in p.get_text():
            nonNASAApplication = p.get_text()
    return nonNASAApplication

def getNASAApplication(bs):
    Ps = bs.find_all("p")
    NASAApplication = ""
    for p in Ps:
        if "POTENTIAL NASA COMMERCIAL APPLICATIONS" in p.get_text():
            NASAApplication = p.get_text()
    return NASAApplication

In [92]:
### Proposal Number
def getProposalNumber(bs):
    trs = bs.find_all("tr")
    proposalNumber = ""
    for tr in trs:
        if "PROPOSAL NUMBER" in tr.get_text():
            proposalNumber = tr.find_all("td")[1].get_text().strip()
    return proposalNumber

### Phase 1 Contract Number
def getPhase1Number(bs):
    trs = bs.find_all("tr")
    Phase1Number = ""
    for tr in trs:
        if "PHASE-I CONTRACT NUMBER" in tr.get_text():
            Phase1Number = tr.find_all("td")[1].get_text().strip()
    return Phase1Number

### Research Topic and project title
def getResearchSubTopicTitle(bs):
    trs = bs.find_all("tr")
    ResearchSubTopicTitle = ""
    for tr in trs:
        if "RESEARCH SUBTOPIC TITLE" in tr.get_text():
            ResearchSubTopicTitle = tr.find_all("td")[1].get_text().strip()
    return ResearchSubTopicTitle


def getProjectTitle(bs):
    trs = bs.find_all("tr")
    ProjectTitle = ""
    for tr in trs:
        if "PROPOSAL TITLE" in tr.get_text():
            ProjectTitle = tr.find_all("td")[1].get_text().strip()
    return ProjectTitle

In [118]:
table = bs.find_all("table")[1]
trs = table.find_all("tr")
for tr in trs:
    if "STATE/ZIP" in tr.get_text():
        smbStateAndZip = tr.find_all("th")[0].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
        riStateAndZip = tr.find_all("th")[1].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
    elif "CITY" in tr.get_text():
        smbCity = tr.find_all("th")[0].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
        riCity = tr.find_all("th")[1].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
    elif "ADDRESS" in tr.get_text():
        smbAddress = tr.find_all("th")[0].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
        riAddress = tr.find_all("th")[1].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
    elif "NAME" in tr.get_text():
        smbName = tr.find_all("th")[0].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
        riName = tr.find_all("th")[1].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
    elif "PHONE" in tr.get_text():
        smbPhone = tr.find_all("th")[0].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
        riPhone = tr.find_all("th")[1].get_text().strip() if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()

In [123]:
table = bs.find_all("table")[1]
trs = table.find_all("tr")
for tr in trs:
        if "STATE/ZIP" in tr.get_text():
            smbStateAndZip = tr.find_all("th")[0].get_text().strip()  if len(tr.find_all("th")) > 0 else tr.find_all("b")[0].get_text().strip()
            riStateAndZip = tr.find_all("th")[1].get_text().strip()
        elif "CITY" in tr.get_text():
            smbCity = tr.find_all("th")[0].get_text().strip()
            riCity = tr.find_all("th")[1].get_text().strip()
        elif "ADDRESS" in tr.get_text():
            smbAddress = tr.find_all("th")[0].get_text().strip()
            riAddress = tr.find_all("th")[1].get_text().strip()
        elif "NAME" in tr.get_text():
            print(tr)
            smbName = tr.find_all("th")[0].get_text().strip()
            riName = tr.find_all("th")[1].get_text().strip()
        elif "PHONE" in tr.get_text():
            smbPhone = tr.find_all("th")[0].get_text().strip()
            riPhone = tr.find_all("th")[1].get_text().strip()

<tr>
<td>NAME:</td>
<td align="LEFT"><b>Advanced Powder Solutions</b></td>
<td>NAME:</td>
<td align="LEFT"><b>UC Davis</b></td>
</tr>


IndexError: list index out of range

In [121]:
### Small Business and Investigator Info
def getSMBAndRIInfo(bs):
    table = bs.find_all("table")[1]
    trs = table.find_all("tr")
    for tr in trs:
        if "STATE/ZIP" in tr.get_text():
            smbStateAndZip = tr.find_all("th")[0].get_text().strip()
            riStateAndZip = tr.find_all("th")[1].get_text().strip()
        elif "CITY" in tr.get_text():
            smbCity = tr.find_all("th")[0].get_text().strip()
            riCity = tr.find_all("th")[1].get_text().strip()
        elif "ADDRESS" in tr.get_text():
            smbAddress = tr.find_all("th")[0].get_text().strip()
            riAddress = tr.find_all("th")[1].get_text().strip()
        elif "NAME" in tr.get_text():
            smbName = tr.find_all("th")[0].get_text().strip()
            riName = tr.find_all("th")[1].get_text().strip()
        elif "PHONE" in tr.get_text():
            smbPhone = tr.find_all("th")[0].get_text().strip()
            riPhone = tr.find_all("th")[1].get_text().strip()
    
    smbAddress = smbAddress + ", " + smbCity + ", " + smbStateAndZip
    riAddress = riAddress + ", " + riCity + ", " + riStateAndZip
    return [smbName, smbAddress, smbPhone, riName, riAddress, riPhone]

In [59]:
def getInvestigatorInfo(bs):
    Ps = bs.find_all("p")
    info = ""
    for p in Ps:
        if "PRINCIPAL INVESTIGATOR/PROJECT MANAGER" in p.get_text():
            info = p.get_text().split('\n')
    name = info[3]
    email = info[4] if len(info) > 4 else ""
    return [name, email]